**<div style='font-size:200%'>Batch Transform using the sm-gluonts entrypoint</div>**

In this notebook, we first register a model artifact into a SageMaker model, then perform a batch evaluation. Optionally, we deregister the model.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import logging
import sagemaker as sm
from sagemaker.mxnet.model import MXNetModel

role: str = sm.get_execution_role()    # When running on SageMaker notebook instance.
sess = sm.Session()
region: str = sess.boto_session.region_name

# Global config

In [ ]:
# I/O S3 paths MUST have trailing '/'
# Good data from yesterday (20200924)
#
# Today's data 20200925 from:
# s3://app01-nvsgisrssr07-be-sh-modelartifactbucketdd4cb-9qo6k8mkeset/planning/inference-runs-NIBR-Cambridge/weekly/inference-230911/preprocess-output/DeepAR-NGB-LSTM-XGB/

bt_input = 's3://BUCKET/BT_INPUT/'
bt_output = 's3://BUCKET/BT_OUTPUT/'
data_json = 'fcast-input.json'

train_model_artifact = "model_s3_FROM_NOTEBOOK_02-hpo-train"

%set_env BT_INPUT=$bt_input
%set_env BT_OUTPUT=$bt_output
%set_env DATA_JSON=$data_json

In [ ]:
train_model_artifact

# Create model

Let SDK auto-generates the model name, so we can safely make this notebook reentrant.

In [ ]:
mxnet_model = MXNetModel(
        model_data=train_model_artifact,
        role=role,
        entry_point='entrypoint.py',
        source_dir='../../src/entrypoint',
        py_version="py3",
        framework_version="1.6.0",
        sagemaker_session=sess,
        container_log_level=logging.DEBUG,
    )

A bit of reverse engineering, to confirm env. vars that the model will end-up using. Will be useful when the time comes where I need to do all these in boto3 or botocore.

In [ ]:
# Before create model
mxnet_model._framework_env_vars()

In [ ]:
# Create model
mxnet_model._create_sagemaker_model(instance_type='ml.m5.xlarge')

In [ ]:
# Model name
mxnet_model.name

In [ ]:
mxnet_model._framework_env_vars()

In [ ]:
# Peek into model's model.tar.gz (which is different from training artifact model.tar.gz).
model_s3 = mxnet_model._framework_env_vars()['SAGEMAKER_SUBMIT_DIRECTORY']
%set_env MODEL_S3=$model_s3
!aws s3 cp $MODEL_S3 - | tar -tzvf -

# Batch Transform

In [ ]:
# Batch Transform
bt = mxnet_model.transformer(
    instance_count=1,
    instance_type='ml.m5.4xlarge',
    strategy='MultiRecord',
    assemble_with='Line',
    output_path=bt_output,
    accept='application/json',
    env={'SAGEMAKER_MODEL_SERVER_TIMEOUT': '3600'},
    max_concurrent_transforms=8,
    max_payload=1,
)

In [ ]:
bt.base_transform_job_name

In [ ]:
bt.transform(
    data=bt_input,
    data_type='S3Prefix',
    content_type='application/json',
    split_type='Line',
    wait=True,
    logs=True,
    #wait=False,
    #logs=False,
)

# Quick check on the results

**<font color="firebrick">NOTE:</font>** if you don't see two exact same numbers, something's wrong, and scream very very loud ASAP!

In [ ]:
!echo $(aws s3 cp ${BT_INPUT}${DATA_JSON} - | wc -l)
!echo $(aws s3 cp ${BT_OUTPUT}${DATA_JSON}.out - | wc -l)

In [ ]:
!aws s3 cp ${BT_OUTPUT}${DATA_JSON}.out - | head -1 | jq

# Delete model

Uncomment and execute cell to "deregister" the model from SageMaker. The inference model artifacts remain untouched in S3.

In [ ]:
#mxnet_model.delete_model()